In [31]:
import pandas as pd
from pandas import Series, DataFrame

In [32]:
titanic_df = pd.read_csv("./titanic_train.csv")

In [33]:
titanic_df.head()
#passenger ID: 탑싀객 일련번호
#Survived: 생존여부(label) : 0: 사망, 1:생존
# Pclass: 선실 등급
# SibSp:  동승한 형제자매 혹은 배우자 인원수
#Parch:  동승한 부모님 또는 자녀수
# Ticket: 티켓 번호
#Embaked: 중간 정착 항구: C: Cherbourg, Q= Queenstown, S=Southhampton

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [34]:
titanic_df.Survived.value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [35]:
#일단 한 번 해보기
# target(label): Survived
# feature: 나머지 컬럼

In [36]:
x_titanic_df = titanic_df.drop('Survived', axis = 1)
y_titanic_df = titanic_df.Survived

In [37]:
#0) 전처리

In [38]:
x_titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Name         891 non-null    object 
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB


In [39]:
# null값 처리, string(object = 문자열) 처리필요 

In [40]:
#문자열 피처를 삭제(문자열이 있으면 학습할 수 없음)
# name, passengerid, Ticket, cabin(객실번호) --> survived와 무관할 것 같아 삭제

x_titanic_df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1, inplace = True)

In [41]:
x_titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       714 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB


In [42]:
#2) 문자열 피처 인코딩(숫자값으로 변환)
# 성별은 문자열이지만 survived(y)에 영향을 미칠 것 같아 숫자로 변환
# 방법 : 성별 피처를 LabelEncoding하여 숫자로 변환
## 알파벳 순서로 인해 female = 0

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
x_titanic_df['Sex'] = le.fit_transform(x_titanic_df.Sex)
x_titanic_df

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,S
1,1,0,38.0,1,0,71.2833,C
2,3,0,26.0,0,0,7.9250,S
3,1,0,35.0,1,0,53.1000,S
4,3,1,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
886,2,1,27.0,0,0,13.0000,S
887,1,0,19.0,0,0,30.0000,S
888,3,0,NaN,1,2,23.4500,S
889,1,1,26.0,0,0,30.0000,C


In [43]:
# 라벨 인코딩 내용(클래스) 확인
le.classes_

array(['female', 'male'], dtype=object)

In [44]:
x_titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    int32  
 2   Age       714 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  889 non-null    object 
dtypes: float64(2), int32(1), int64(3), object(1)
memory usage: 45.4+ KB


In [45]:
#Embarked 피처를 onehot Encoding하여 숫자로 변환 (onehot인코딩으로 바꿔보기)
# --> [0 1 0] | [1 0 0] => 집합 중 하나만 1로 만드는 것
# Enbarked에는 Null값이 2개 있음 - 처리요망

x_titanic_df = pd.get_dummies(x_titanic_df)
x_titanic_df

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,3,1,22.0,1,0,7.2500,0,0,1
1,1,0,38.0,1,0,71.2833,1,0,0
2,3,0,26.0,0,0,7.9250,0,0,1
3,1,0,35.0,1,0,53.1000,0,0,1
4,3,1,35.0,0,0,8.0500,0,0,1
...,...,...,...,...,...,...,...,...,...
886,2,1,27.0,0,0,13.0000,0,0,1
887,1,0,19.0,0,0,30.0000,0,0,1
888,3,0,NaN,1,2,23.4500,0,0,1
889,1,1,26.0,0,0,30.0000,1,0,0


In [46]:
# 결측치 확인
x_titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    int64  
 1   Sex         891 non-null    int32  
 2   Age         714 non-null    float64
 3   SibSp       891 non-null    int64  
 4   Parch       891 non-null    int64  
 5   Fare        891 non-null    float64
 6   Embarked_C  891 non-null    uint8  
 7   Embarked_Q  891 non-null    uint8  
 8   Embarked_S  891 non-null    uint8  
dtypes: float64(2), int32(1), int64(3), uint8(3)
memory usage: 41.0 KB


In [47]:
#3) NaN  결측치 채워넣기 (Age 항목 714개)
x_titanic_df.Age.fillna(x_titanic_df.Age.mean(), inplace = True)

In [48]:
x_titanic_df.info() #Age 값의 결측치 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    int64  
 1   Sex         891 non-null    int32  
 2   Age         891 non-null    float64
 3   SibSp       891 non-null    int64  
 4   Parch       891 non-null    int64  
 5   Fare        891 non-null    float64
 6   Embarked_C  891 non-null    uint8  
 7   Embarked_Q  891 non-null    uint8  
 8   Embarked_S  891 non-null    uint8  
dtypes: float64(2), int32(1), int64(3), uint8(3)
memory usage: 41.0 KB


In [49]:
# Age columms을 Age_group컬럼으로 변환
# 0 ~ 5 : Baby, 6~12: Child, 13~18: Teenager, 19~25: Student, 26~35: Young Adult, 36~60: Adult, 61~: elderly
x_titanic_df.Age.value_counts()

29.699118    177
24.000000     30
22.000000     27
18.000000     26
28.000000     25
            ... 
36.500000      1
55.500000      1
0.920000       1
23.500000      1
74.000000      1
Name: Age, Length: 89, dtype: int64

In [50]:
import numpy as np
x_titanic_df['Age_group'] = np.digitize(x_titanic_df.Age, [0, 3, 13, 19, 26, 36, 61])
x_titanic_df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Age_group
0,3,1,22.0,1,0,7.2500,0,0,1,4
1,1,0,38.0,1,0,71.2833,1,0,0,6
2,3,0,26.0,0,0,7.9250,0,0,1,5
3,1,0,35.0,1,0,53.1000,0,0,1,5
4,3,1,35.0,0,0,8.0500,0,0,1,5


In [51]:
x_titanic_df.drop('Age', axis = 1, inplace = True)
x_titanic_df

,Pclass,Sex,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Age_group
0,3,1,1,0,7.2500,0,0,1,4
1,1,0,1,0,71.2833,1,0,0,6
2,3,0,0,0,7.9250,0,0,1,5
3,1,0,1,0,53.1000,0,0,1,5
4,3,1,0,0,8.0500,0,0,1,5
...,...,...,...,...,...,...,...,...,...
886,2,1,0,0,13.0000,0,0,1,5
887,1,0,0,0,30.0000,0,0,1,4
888,3,0,1,2,23.4500,0,0,1,5
889,1,1,0,0,30.0000,1,0,0,5


In [66]:
x_titanic_df.Age_group.value_counts().sort_index()

1     24
2     45
3     70
4    162
5    373
6    195
7     22
Name: Age_group, dtype: int64

여기까지 EDA 과정 <hr>

In [80]:
#1) 학습용, 테스트용 데이터 분리
from sklearn.model_selection import train_test_split
# test_size = default 0.25

x_train, x_test, y_train, y_test = train_test_split(x_titanic_df, y_titanic_df, random_state = 213)

In [84]:
#2) 결정트리 객체 생성
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()

In [86]:
#3) 학습
dt.fit(x_train, y_train)

DecisionTreeClassifier()

In [91]:
#4) 평가 (정확도)
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = dt.predict(x_train)
train_score = accuracy_score(y_train, y_pred)

y_pred = dt.predict(x_test)
test_score = accuracy_score(y_test, y_pred)
print(train_score, test_score)
# 결과 : 과적합

0.9505988023952096 0.8340807174887892


In [92]:
# 같은 코드
train_score = dt.score(x_train, y_train)
test_score = dt.score(x_test, y_test)
print(train_score, test_score)

0.9505988023952096 0.8340807174887892


In [57]:
# 평가 점수가 낮아 전처리를 세분화해보기로 함


In [108]:
#결정트리 최적화 - 하이퍼파라미터 튜닝 & 교차검증 -: GridSearchCV()
from sklearn.model_selection import GridSearchCV

dt_params = {
    'max_depth' : [3, 5, 7],
    'min_samples_split' : [5, 10, 15],
    'splitter' : ['best']
}   # max_depth, min_samples_split의 list는 분석가가 조절해가기
# splitter이 'random'이라 점수가 다름

grid_dt = GridSearchCV(dt, param_grid = dt_params, cv = 5)
grid_dt.fit(x_train, y_train)


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [3, 5, 7],
                         'min_samples_split': [5, 10, 15],
                         'splitter': ['best']})

In [109]:
grid_dt.best_params_

{'max_depth': 5, 'min_samples_split': 10, 'splitter': 'best'}

In [110]:
grid_dt.score(x_train, y_train), grid_dt.score(x_test, y_test)

(0.8592814371257484, 0.7937219730941704)

In [111]:
#다른 분류 알고리듬으로 예측하기

from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(solver = 'lbfgs', max_iter = 500)
lr_clf.fit(x_train, y_train)

print(f'train_score : {lr_clf.score(x_train, y_train)}')
print(f'test_score : {lr_clf.score(x_test, y_test)}')

train_score : 0.8218562874251497
test_score : 0.7847533632286996


In [113]:
from sklearn.svm import SVC # support 벡터 머신

svm_clf = SVC(gamma = 'scale')
svm_clf.fit(x_train, y_train)
print(f'train_score : {svm_clf.score(x_train, y_train)}')
print(f'test_score : {svm_clf.score(x_test, y_test)}')

train_score : 0.6916167664670658
test_score : 0.6367713004484304


데이터 셋은 하나인데 여러 알고리즘을 사용해서 그 중 하나를 제출하면 됨

In [60]:
# 결정 트리 시각화

In [ ]:
from sklearn.tree import plot_tree

import matplotlib

In [61]:
%matplotlib inline

In [62]:
#결정트리 시각화

In [63]:
#피처와 라벨 정보를 기입하여 가독성 향상


In [64]:
# 간단한 트리 생성

In [65]:
#결정트리 그래프를 파일로 받아 Pdf 저장
